In [102]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from sklearn.model_selection import KFold

# 1.Chargement de data et analyse

In [103]:
#Récupération des datas du fichier CSV

In [104]:
Ligue1=pd.read_csv("Data/RegroupSortie.csv",sep=';',encoding = "ISO-8859-1")

In [105]:
#Affichage du dataframe

In [106]:
Ligue1

,Team,Position,NbMatchGagne,NbMatchNul,NbMatchPerdu,NbButMis,NbButEnc,Forme5,Forme4,Forme3,...,Forme4_away,Forme3_away,Forme2_away,Forme1_away,Sante_away,home_score,away_score,saison,Journee,result
0,AS MONACO,6.0,3.0,0.0,2.0,9.0,5.0,1.0,5.0,1.0,...,1.0,3.0,3.0,5.0,10.0,0,0,1993-1994,6,0
1,A.S. CANNES,1.0,3.0,2.0,0.0,8.0,5.0,5.0,5.0,3.0,...,3.0,5.0,1.0,1.0,10.0,1,1,1993-1994,6,0
2,AS SAINT-ÉTIENNE,13.0,1.0,2.0,2.0,5.0,5.0,1.0,1.0,5.0,...,5.0,3.0,1.0,5.0,10.0,0,0,1993-1994,6,0
3,FC NANTES,4.0,3.0,1.0,1.0,5.0,2.0,5.0,5.0,5.0,...,1.0,5.0,1.0,1.0,10.0,0,0,1993-1994,6,0
4,AJ AUXERRE,9.0,1.0,3.0,1.0,4.0,3.0,3.0,5.0,1.0,...,5.0,1.0,5.0,5.0,10.0,3,2,1993-1994,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8883,CLERMONT FOOT 63,12.0,8.0,6.0,10.0,24.0,33.0,3.0,3.0,1.0,...,1.0,5.0,1.0,5.0,10.0,1,1,2022-2023,25,0
8884,FC NANTES,13.0,6.0,10.0,8.0,25.0,29.0,3.0,1.0,5.0,...,5.0,1.0,1.0,5.0,10.0,0,1,2022-2023,25,2
8885,STADE DE REIMS,10.0,7.0,13.0,4.0,30.0,26.0,3.0,5.0,3.0,...,5.0,1.0,5.0,1.0,10.0,3,0,2022-2023,25,1
8886,AS MONACO,3.0,15.0,5.0,4.0,53.0,31.0,3.0,5.0,5.0,...,5.0,5.0,5.0,3.0,10.0,0,3,2022-2023,25,2


In [107]:
#Pourcentage de match nul

In [108]:
print((sum(Ligue1["result"]==0)/len(Ligue1))*100)

28.960396039603957


In [109]:
#Pourcentage de victoire à domicile

In [110]:
print((sum(Ligue1["result"]==1)/len(Ligue1))*100)

46.005850585058504


In [111]:
#Pourcentage de victoire à l'éxtérieur

In [112]:
print((sum(Ligue1["result"]==2)/len(Ligue1))*100)

25.033753375337536


# 2.Nettoyage

In [113]:
#Supprime les matchs annuler ou reportés ceux avec un score de 99

In [114]:
Ligue1.drop(Ligue1[Ligue1['home_score']==99].index,inplace=True)

In [115]:
#Ligue1

In [116]:
#Supression de colones inutiles

In [117]:
Ligue1=Ligue1.drop(['Sante','Team','Team_away','Sante_away','saison','Journee','home_score','away_score'],axis=1)

In [118]:
#Ligue1

In [119]:
#Supprimer les champs vide

In [120]:
Ligue1.dropna(axis=0,inplace=True)

In [121]:
#Ligue1

In [122]:
#inversion du classement home team

In [123]:
#Ligue1['Position']=1/(Ligue1['Position'])

In [124]:
Ligue1['Position']=(21)-(Ligue1['Position'])

In [125]:
#inversion du classement away team

In [126]:
#Ligue1['Position_away']=1/(Ligue1['Position_away'])

In [127]:
Ligue1['Position_away']=(21)-(Ligue1['Position_away'])

In [128]:
#Ligue1.tail(22)

# 3.Convertion du dataframe en tableau

In [129]:
#Met y sous forme d'un tableau
y=Ligue1[['result']].values

In [130]:
#Met X sous forme de tableau

In [131]:
X=Ligue1.drop('result',axis=1).values
X.shape

(8353, 22)

# 4.Cross validation


In [132]:
for train_index,test_index in KFold(3,shuffle=True, random_state=0).split(X):
    X_train,X_test=X[train_index],X[test_index]
    y_train,y_test=y[train_index],y[test_index]

# 5.Création d'un réseau de Neurone Artificiel

In [133]:
model = Sequential()

model.add(Dense(32, input_dim=22))
model.add(Activation('relu'))
model.add(Dense(12))
model.add(Dense(12))


model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                736       
                                                                 
 activation_2 (Activation)   (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 12)                396       
                                                                 
 dense_6 (Dense)             (None, 12)                156       
                                                                 
 dense_7 (Dense)             (None, 3)                 39        
                                                                 
 activation_3 (Activation)   (None, 3)                 0         
                                                                 
Total params: 1,327
Trainable params: 1,327
Non-traina

In [134]:
#Entraînement du model

In [135]:
model.fit(X_train,y_train,epochs=100,batch_size=32,verbose=1)

Epoch 1/100
175/175 [==============================] - 0s 845us/step - loss: 1.3268 - sparse_categorical_accuracy: 0.3954
Epoch 2/100
175/175 [==============================] - 0s 833us/step - loss: 1.0785 - sparse_categorical_accuracy: 0.4498
Epoch 3/100
175/175 [==============================] - 0s 845us/step - loss: 1.0628 - sparse_categorical_accuracy: 0.4532
Epoch 4/100
175/175 [==============================] - 0s 833us/step - loss: 1.0576 - sparse_categorical_accuracy: 0.4557
Epoch 5/100
175/175 [==============================] - 0s 839us/step - loss: 1.0603 - sparse_categorical_accuracy: 0.4620
Epoch 6/100
175/175 [==============================] - 0s 833us/step - loss: 1.0444 - sparse_categorical_accuracy: 0.4647
Epoch 7/100
175/175 [==============================] - 0s 856us/step - loss: 1.0424 - sparse_categorical_accuracy: 0.4706
Epoch 8/100
175/175 [==============================] - 0s 856us/step - loss: 1.0355 - sparse_categorical_accuracy: 0.4757
Epoch 9/100
175/175 [===

175/175 [==============================] - 0s 834us/step - loss: 1.0037 - sparse_categorical_accuracy: 0.4990
Epoch 68/100
175/175 [==============================] - 0s 833us/step - loss: 1.0080 - sparse_categorical_accuracy: 0.4999
Epoch 69/100
175/175 [==============================] - 0s 828us/step - loss: 1.0040 - sparse_categorical_accuracy: 0.5039
Epoch 70/100
175/175 [==============================] - 0s 828us/step - loss: 1.0037 - sparse_categorical_accuracy: 0.5004
Epoch 71/100
175/175 [==============================] - 0s 828us/step - loss: 1.0039 - sparse_categorical_accuracy: 0.5019
Epoch 72/100
175/175 [==============================] - 0s 833us/step - loss: 1.0043 - sparse_categorical_accuracy: 0.5087
Epoch 73/100
175/175 [==============================] - 0s 828us/step - loss: 1.0067 - sparse_categorical_accuracy: 0.5076
Epoch 74/100
175/175 [==============================] - 0s 839us/step - loss: 1.0057 - sparse_categorical_accuracy: 0.5044
Epoch 75/100
175/175 [=======

In [136]:
#Evaluation du model

In [137]:
test_loss, test_accuracy= model.evaluate(X_test,y_test)

87/87 [==============================] - 0s 698us/step - loss: 1.0550 - sparse_categorical_accuracy: 0.4716


# 6.Prédiction

In [138]:
#Prédiction / Chargement de la data à prédire

In [139]:
match_futur=pd.read_csv("Data/Prevision24.csv",sep=';',encoding = "ISO-8859-1")

In [140]:
match_a_predir=match_futur

In [141]:
#Supression des noms d'équipe

In [142]:
match_a_predir=match_a_predir.drop('Team',axis=1)

In [143]:
match_a_predir=match_a_predir.drop('Team_away',axis=1)

In [144]:
#inversion des postions

In [145]:
match_a_predir['Position']=(21)-(match_a_predir['Position'])

In [146]:
match_a_predir['Position_away']=(21)-(match_a_predir['Position_away'])

In [147]:
X_pred=match_a_predir.values

In [148]:
prediction=model.predict(X_pred)

1/1 [==============================] - 0s 48ms/step


In [149]:
#prediction

In [150]:
i=0
for pred in prediction:
    print ( match_futur["Team"][i]," VS ",match_futur["Team_away"][i])
    if (np.argmax(pred))==0:
        print("Match nul\n")
    elif (np.argmax(pred))==1:
        print("Vicoire pour ",match_futur["Team"][i],"\n")
    elif (np.argmax(pred))==2:
        print("Vicoire pour ",match_futur["Team_away"][i],"\n")
    #Affiche la catégorie de la prédiction
    #print (np.argmax(pred))
    i=i+1

AJ AUXERRE  VS  OLYMPIQUE LYONNAIS
Vicoire pour  OLYMPIQUE LYONNAIS 

OGC NICE  VS  STADE DE REIMS
Vicoire pour  OGC NICE 

RC STRASBOURG ALSACE  VS  ANGERS SCO
Match nul

PARIS SAINT-GERMAIN  VS  LOSC LILLE
Vicoire pour  PARIS SAINT-GERMAIN 

STADE BRESTOIS 29  VS  AS MONACO
Vicoire pour  AS MONACO 

FC LORIENT  VS  AC AJACCIO
Vicoire pour  FC LORIENT 

STADE RENNAIS FC  VS  CLERMONT FOOT 63
Vicoire pour  STADE RENNAIS FC 

ESTAC TROYES  VS  MONTPELLIER HÉRAULT SC
Vicoire pour  MONTPELLIER HÉRAULT SC 

RC LENS  VS  FC NANTES
Vicoire pour  RC LENS 

TOULOUSE FC  VS  OLYMPIQUE DE MARSEILLE
Vicoire pour  OLYMPIQUE DE MARSEILLE 

